## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---


In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt




## Camera Callibration

Callibration of the camera is done using chessboard images provided in the `camera_cal` folder. 

We also use object possitions that have been provided for each image.

In [11]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9, 3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

img = cv2.imread('camera_cal/calibration2.jpg')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)



ret, corners = cv2.findChessboardCorners(img_gray, (9,5), None)

print(corners)

plt.imshow(img_gray)

None


In [21]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')


board_size =  (9,6)


# Step through the list and search for chessboard corners
for fname in images:
    
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, board_size, None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, board_size, corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)
        print(fname)

cv2.destroyAllWindows()

camera_cal/calibration6.jpg
camera_cal/calibration7.jpg
camera_cal/calibration3.jpg
camera_cal/calibration2.jpg
camera_cal/calibration20.jpg
camera_cal/calibration19.jpg
camera_cal/calibration18.jpg
camera_cal/calibration15.jpg
camera_cal/calibration14.jpg
camera_cal/calibration16.jpg
camera_cal/calibration17.jpg
camera_cal/calibration13.jpg
camera_cal/calibration12.jpg
camera_cal/calibration10.jpg
camera_cal/calibration11.jpg
camera_cal/calibration9.jpg
camera_cal/calibration8.jpg


In [24]:
# callibrate camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_gray.shape[::-1], None, None)


[[1.15777818e+03 0.00000000e+00 6.67113857e+02]
 [0.00000000e+00 1.15282217e+03 3.86124583e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [27]:
# Check how undistorted image looks

img = cv2.imread('camera_cal/calibration2.jpg')
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

dst = cv2.undistort(img, mtx, dist, None, mtx)
plt.imshow(dst)